In [11]:
install.packages("hdf5r")
install.packages("data.table")
install.packages("raster")
install.packages("ggplot2")

In [28]:
library(hdf5r)
library(data.table)
library(raster)
library(ggplot2)
library(stringr)
library(parallel)
setwd("/projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Senegal_Workshop/")
source("GEDI_FUN.R")

In [25]:
#get a list of GEDI L2A files
L2A.ls<-list.files("/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data",pattern="*.h5", recursive = TRUE, full.names = TRUE)
L2A.ls<-L2A.ls[!str_detect(L2A.ls,pattern="sha256")]

#Pick one!
gedi.file<-L2A.ls[1]

#read in the GEDI L2A file
L2A.h5<-readLevel2A(gedi.file)

#Let's look at the GEDI data structure
list.datasets(L2A.h5[["BEAM0000"]])
print(L2A.ls)
#now we can read all GEDI files into a list and combine

[1] "agbd"                                     
  [2] "agbd_pi_lower"                            
  [3] "agbd_pi_upper"                            
  [4] "agbd_prediction/agbd_a1"                  
  [5] "agbd_prediction/agbd_a10"                 
  [6] "agbd_prediction/agbd_a2"                  
  [7] "agbd_prediction/agbd_a3"                  
  [8] "agbd_prediction/agbd_a4"                  
  [9] "agbd_prediction/agbd_a5"                  
 [10] "agbd_prediction/agbd_a6"                  
 [11] "agbd_prediction/agbd_pi_lower_a1"         
 [12] "agbd_prediction/agbd_pi_lower_a10"        
 [13] "agbd_prediction/agbd_pi_lower_a2"         
 [14] "agbd_prediction/agbd_pi_lower_a3"         
 [15] "agbd_prediction/agbd_pi_lower_a4"         
 [16] "agbd_prediction/agbd_pi_lower_a5"         
 [17] "agbd_prediction/agbd_pi_lower_a6"         
 [18] "agbd_prediction/agbd_pi_upper_a1"         
 [19] "agbd_prediction/agbd_pi_upper_a10"        
 [20] "agbd_prediction/agbd_pi_upper_a2"         
 [21] "agbd_prediction/agbd_pi_upper_a3"         
 [22] "agbd_prediction/agbd_pi_upper_a4"         
 [23] "agbd_prediction/agbd_pi_upper_a5"         
 [24] "agbd_prediction/agbd_pi_upper_a6"         
 [25] "agbd_prediction/agbd_se_a1"               
 [26] "agbd_prediction/agbd_se_a10"              
 [27] "agbd_prediction/agbd_se_a2"               
 [28] "agbd_prediction/agbd_se_a3"               
 [29] "agbd_prediction/agbd_se_a4"               
 [30] "agbd_prediction/agbd_se_a5"               
 [31] "agbd_prediction/agbd_se_a6"               
 [32] "agbd_prediction/agbd_t_a1"                
 [33] "agbd_prediction/agbd_t_a10"               
 [34] "agbd_prediction/agbd_t_a2"                
 [35] "agbd_prediction/agbd_t_a3"                
 [36] "agbd_prediction/agbd_t_a4"                
 [37] "agbd_prediction/agbd_t_a5"                
 [38] "agbd_prediction/agbd_t_a6"                
 [39] "agbd_prediction/agbd_t_pi_lower_a1"       
 [40] "agbd_prediction/agbd_t_pi_lower_a10"      
 [41] "agbd_prediction/agbd_t_pi_lower_a2"       
 [42] "agbd_prediction/agbd_t_pi_lower_a3"       
 [43] "agbd_prediction/agbd_t_pi_lower_a4"       
 [44] "agbd_prediction/agbd_t_pi_lower_a5"       
 [45] "agbd_prediction/agbd_t_pi_lower_a6"       
 [46] "agbd_prediction/agbd_t_pi_upper_a1"       
 [47] "agbd_prediction/agbd_t_pi_upper_a10"      
 [48] "agbd_prediction/agbd_t_pi_upper_a2"       
 [49] "agbd_prediction/agbd_t_pi_upper_a3"       
 [50] "agbd_prediction/agbd_t_pi_upper_a4"       
 [51] "agbd_prediction/agbd_t_pi_upper_a5"       
 [52] "agbd_prediction/agbd_t_pi_upper_a6"       
 [53] "agbd_prediction/agbd_t_se_a1"             
 [54] "agbd_prediction/agbd_t_se_a10"            
 [55] "agbd_prediction/agbd_t_se_a2"             
 [56] "agbd_prediction/agbd_t_se_a3"             
 [57] "agbd_prediction/agbd_t_se_a4"             
 [58] "agbd_prediction/agbd_t_se_a5"             
 [59] "agbd_prediction/agbd_t_se_a6"             
 [60] "agbd_prediction/algorithm_run_flag_a1"    
 [61] "agbd_prediction/algorithm_run_flag_a10"   
 [62] "agbd_prediction/algorithm_run_flag_a2"    
 [63] "agbd_prediction/algorithm_run_flag_a3"    
 [64] "agbd_prediction/algorithm_run_flag_a4"    
 [65] "agbd_prediction/algorithm_run_flag_a5"    
 [66] "agbd_prediction/algorithm_run_flag_a6"    
 [67] "agbd_prediction/l2_quality_flag_a1"       
 [68] "agbd_prediction/l2_quality_flag_a10"      
 [69] "agbd_prediction/l2_quality_flag_a2"       
 [70] "agbd_prediction/l2_quality_flag_a3"       
 [71] "agbd_prediction/l2_quality_flag_a4"       
 [72] "agbd_prediction/l2_quality_flag_a5"       
 [73] "agbd_prediction/l2_quality_flag_a6"       
 [74] "agbd_prediction/l4_quality_flag_a1"       
 [75] "agbd_prediction/l4_quality_flag_a10"      
 [76] "agbd_prediction/l4_quality_flag_a2"       
 [77] "agbd_prediction/l4_quality_flag_a3"       
 [78] "agbd_prediction/l4_quality_flag_a4"       
 [79] "agbd_prediction/l4_quality_flag_a5"       
 [80] "agbd_prediction/l4_quality_flag_a6"       
 [

 [1] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2019167060146_O02879_02_T02321_02_002_02_V002.h5"
 [2] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2019212222100_O03588_03_T04378_02_002_02_V002.h5"
 [3] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2020120102830_O07814_03_T04378_02_002_02_V002.h5"
 [4] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2020292142200_O10483_03_T07224_02_002_02_V002.h5"
 [5] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021091205902_O13045_03_T10070_02_002_02_V002.h5"
 [6] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021169035935_O14243_02_T10859_02_002_02_V002.h5"
 [7] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021173022634_O14304_02_T09283_02_002_02_V002.h5"
 [8] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021218184855_O15012_03_T10070_02_002_02_V002.h5"
 [9] "/p

In [26]:
#we use a lapply function to do this as it can easily be made parallel
L2A.ls.all<-lapply(L2A.ls, function(x){
  L2A.m<-NULL
  L2A.h5<-readLevel2A(x)
  try(L2A.m<-getLevel2A(L2A.h5), silent = TRUE)
  return(L2A.m)
})

In [38]:
L2A.ls
L2A.h5<-readLevel2A('/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2019167060146_O02879_02_T02321_02_002_02_V002.h5')
L2A.m<-getLevel2A(L2A.h5)

[1] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2019167060146_O02879_02_T02321_02_002_02_V002.h5"
 [2] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2019212222100_O03588_03_T04378_02_002_02_V002.h5"
 [3] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2020120102830_O07814_03_T04378_02_002_02_V002.h5"
 [4] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2020292142200_O10483_03_T07224_02_002_02_V002.h5"
 [5] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021091205902_O13045_03_T10070_02_002_02_V002.h5"
 [6] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021169035935_O14243_02_T10859_02_002_02_V002.h5"
 [7] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021173022634_O14304_02_T09283_02_002_02_V002.h5"
 [8] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021218184855_O15012_03_T10070_02_002_02_V002.h5"
 [9] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2021274203848_O15881_03_T07224_02_002_02_V002.h5"
[10] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2022215193351_O20625_03_T08494_02_003_02_V002.h5"
[11] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2022295120050_O21861_03_T10070_02_003_01_V002.h5"
[12] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2023068052900_O23997_03_T04378_02_003_01_V002.h5"
[13] "/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/Data/GEDI04_A_2023072035526_O24058_03_T07224_02_003_01_V002.h5"

ERROR: Error in `[[.H5Group`(level2a_i, "rh"): An object with name rh does not exist in this group


In [33]:
#lets actually do that! Make it parallel:

# Calculate the number of cores
no_cores <- detectCores() - 1

# Initiate cluster
cl <- makeCluster(no_cores)

L2A.ls.all<-
  lapply(cl, L2A.ls, function(x){   
    library(hdf5r)
    library(data.table)
    library(raster)    
    source("R/GEDI_FUN.R")    
    L2A.m<-NULL
    try(L2A.h5<-readLevel2A(x), silent = TRUE)
    try(L2A.m<-getLevel2A(L2A.h5), silent = TRUE)
    return(L2A.m)
  })
# stopCluster(cl)

#combine the list into a single long data frame with the rbind command
L2A.all<-do.call(rbind,L2A.ls.all)

#Make a spatial object from the DF for subsetting and extraction. Don't forget the CRS!
L2A.sp<-SpatialPointsDataFrame(coords=cbind(L2A.all$lon_lowestmode,L2A.all$lat_lowestmode),
                               data=L2A.all,
                               proj4string = crs(raster()))
#we bring in our extent polygon and use it for subsetting.
extent.poly<-rgdal::readOGR("/projects/my-public-bucket/Data/NASA_CMS_2023/SENEGAL/GGNP_Nigeria.gpkg")

ERROR: Error in get(as.character(FUN), mode = "function", envir = envir): object 'L2A.ls' of mode 'function' was not found


In [ ]:
#Why do we use extract? The clip function will not accept geometry, 
# so the square extent is the only thing it will clip to. If you want to use 
# complex geometry, use this method.
ext.ex<-extract(extent.poly, L2A.sp)

#add extracted values to the GEDI data
L2A.sp@data$ext.flag<-ext.ex[,2]

#now subset by the identified extent flag
L2A.sp<-L2A.sp[!is.na(L2A.sp$ext.flag),]

#we also need to convert the shot number column to an integer for writing purposes
L2A.sp$shot_number<-as.numeric(L2A.sp$shot_number)

#write the file!
writeOGR(L2A.sp, "GEDI_L2A.shp", layer='shot_number', driver="ESRI Shapefile")

#We can explore the GEDI data in QGIS to determine how to QAQC


###### QAQC Subsetting ############

#now let's subset based on our observations
L2A.q1.sp<-L2A.sp[L2A.sp$degrade_flag==0&
                    L2A.sp$quality_flag==1,]

#write the file!
writeOGR(L2A.q1.sp, "GEDI_L2A_high_quality.shp", layer='shot_number', driver="ESRI Shapefile")

#now we can take a look at some of our GEDI values
gedi.sub<-L2A.q1.sp@data[L2A.q1.sp@data$delta_time <= (104334330.891843348741531) &
                           L2A.q1.sp@data$delta_time >= (104334330.891843348741531-100) &
                           L2A.q1.sp@data$beam == "BEAM0101",]

#where are we? Let's look at an interactive map
library(leaflet)
pal <- colorNumeric(
  palette = "Blues",
  domain = gedi.sub$rh100)

m1<-leaflet() %>%
  addCircleMarkers(gedi.sub$lon_lowestmode,
                   gedi.sub$lat_lowestmode,
                   radius = 1,
                   opacity = 1,
                   color = pal(gedi.sub$rh100))  %>%
  addScaleBar(options = list(imperial = FALSE)) %>%
  addProviderTiles(providers$Esri.WorldImagery)  %>%
  addLegend(colors = "blue", labels= c("GEDI Shots"),
            title ="GEDI Level2A") 
m1

library(ggplot2)
ggplot(gedi.sub,
       aes(x=delta_time,y=elev_lowestmode))+
  geom_ribbon(aes(ymin=elev_lowestmode,ymax=rh100+elev_lowestmode),
              fill="forestgreen", alpha=0.5)+
  geom_path(aes(y=rh100+elev_lowestmode), color="forestgreen")+
  geom_path(color="black", linewidth=0.4)+
  theme_bw()+theme(panel.grid = element_blank())


#Let's compare to another 7 m resolution canopy height product from ALS
CHM.dd<-raster("Data/ALS/CHM_dd.tif")
chm.ex<-raster::extract(CHM.dd, 
                        L2A.q1.sp,
                        buffer = 25/2/111139, fun='mean')
#add extracted values to the GEDI data
L2A.q1.sp$CHM<-chm.ex

gedi.sub<-L2A.q1.sp@data[L2A.q1.sp@data$delta_time <= (104334330.891843348741531) &
                           L2A.q1.sp@data$delta_time >= (104334330.891843348741531-10) &
                           L2A.q1.sp@data$beam == "BEAM0101",]

library(ggplot2)
ggplot(gedi.sub,
       aes(x=delta_time,y=0))+
  geom_ribbon(aes(ymin=0,ymax=rh98),
              fill="forestgreen", alpha=0.5)+
  geom_path(aes(y=rh98), color="forestgreen")+
  geom_path(color="black", linewidth=0.4)+
  geom_path(aes(y=CHM),color="purple")
theme_bw()+theme(panel.grid = element_blank())


L2A.all.sub<-L2A.q1.sp[!is.na(L2A.q1.sp$CHM)&
                         L2A.q1.sp$quality_flag==1&
                         L2A.q1.sp$sensitivity>0.95,]

library(ggplot2)
ggplot(L2A.all.sub@data, aes(rh98, CHM))+
  geom_point(aes(color=sensitivity))+
  facet_wrap(~selected_algorithm)+
  geom_abline(slope=1,intercept=0)


L2A.all.sub<-L2A.all[!is.na(L2A.all$CHM)&
                       L2A.all$quality_flag==1
                     # L2A.all$degrade_flag==0
                     ,]

plot(L2A.all.sub$rh98,L2A.all.sub$CHM)
abline(0,1, col="blue")




GLAD.CHM<-raster("Data/Aux/GLAD/GLAD_GEDI.tif")
GLAD.CHM.ex<-raster::extract(GLAD.CHM, 
                             L2A.q1.sp,
                             buffer = 25/2/111139, fun='mean')

L2A.q1.sp$GLAD.CHM<-GLAD.CHM.ex

lm(rh100~GLAD.CHM-1, L2A.q1.sp@data[L2A.q1.sp$GLAD.CHM<30&
                                      L2A.q1.sp$GLAD.CHM>4&
                                      L2A.q1.sp$rh100>=5&
                                      L2A.q1.sp$rh100<=30&
                                      L2A.q1.sp$selected_algorithm==2,])

GLAD.CHM[GLAD.CHM>40]<-NA
plot(GLAD.CHM*1.411)

chm.df<-as.data.frame(rasterToPoints(CHM.dd))
chm.ag.mean<-raster::rasterize(chm.df[,1:2], GLAD.CHM, field=chm.df$chm_map, fun=mean)
chm.ag.sd<-raster::rasterize(chm.df[,1:2], GLAD.CHM, field=chm.df$chm_map, fun=sd)
plot(stack(chm.ag.mean,chm.ag.sd))

chm_comp<-stack(GLAD.CHM,chm.ag.mean)
chm_comp.df<-as.data.frame(rasterToPoints(chm_comp))
colnames(chm_comp.df)[4]<-"chm.ag"

ggplot(chm_comp.df[!is.na(chm_comp.df$chm.ag&
                            chm_comp.df$chm.ag<35),], 
       aes(chm.ag,GLAD_GEDI*1.411))+
  geom_jitter(size=0.2,height=0.7, alpha=0.05)+
  # geom_hex(bins=100)+
  geom_smooth()+
  geom_abline(slope=1,intercept=0)

ggplot(L2A.q1.sp@data[L2A.q1.sp$GLAD.CHM<30&
                        L2A.q1.sp$GLAD.CHM>4&
                        L2A.q1.sp$rh100>=5&
                        L2A.q1.sp$rh100<=30 ,], aes(GLAD.CHM,rh100))+
  geom_jitter(aes(color=sensitivity), size=0.5,width=0.5)+
  # geom_hex(bins=100)+
  facet_wrap(~selected_algorithm)+
  geom_smooth(method="lm", formula=y~x-1)+
  geom_abline(slope=1,intercept=0)

[1] "agbd"                                     
  [2] "agbd_pi_lower"                            
  [3] "agbd_pi_upper"                            
  [4] "agbd_prediction/agbd_a1"                  
  [5] "agbd_prediction/agbd_a10"                 
  [6] "agbd_prediction/agbd_a2"                  
  [7] "agbd_prediction/agbd_a3"                  
  [8] "agbd_prediction/agbd_a4"                  
  [9] "agbd_prediction/agbd_a5"                  
 [10] "agbd_prediction/agbd_a6"                  
 [11] "agbd_prediction/agbd_pi_lower_a1"         
 [12] "agbd_prediction/agbd_pi_lower_a10"        
 [13] "agbd_prediction/agbd_pi_lower_a2"         
 [14] "agbd_prediction/agbd_pi_lower_a3"         
 [15] "agbd_prediction/agbd_pi_lower_a4"         
 [16] "agbd_prediction/agbd_pi_lower_a5"         
 [17] "agbd_prediction/agbd_pi_lower_a6"         
 [18] "agbd_prediction/agbd_pi_upper_a1"         
 [19] "agbd_prediction/agbd_pi_upper_a10"        
 [20] "agbd_prediction/agbd_pi_upper_a2"         
 [21] "agbd_prediction/agbd_pi_upper_a3"         
 [22] "agbd_prediction/agbd_pi_upper_a4"         
 [23] "agbd_prediction/agbd_pi_upper_a5"         
 [24] "agbd_prediction/agbd_pi_upper_a6"         
 [25] "agbd_prediction/agbd_se_a1"               
 [26] "agbd_prediction/agbd_se_a10"              
 [27] "agbd_prediction/agbd_se_a2"               
 [28] "agbd_prediction/agbd_se_a3"               
 [29] "agbd_prediction/agbd_se_a4"               
 [30] "agbd_prediction/agbd_se_a5"               
 [31] "agbd_prediction/agbd_se_a6"               
 [32] "agbd_prediction/agbd_t_a1"                
 [33] "agbd_prediction/agbd_t_a10"               
 [34] "agbd_prediction/agbd_t_a2"                
 [35] "agbd_prediction/agbd_t_a3"                
 [36] "agbd_prediction/agbd_t_a4"                
 [37] "agbd_prediction/agbd_t_a5"                
 [38] "agbd_prediction/agbd_t_a6"                
 [39] "agbd_prediction/agbd_t_pi_lower_a1"       
 [40] "agbd_prediction/agbd_t_pi_lower_a10"      
 [41] "agbd_prediction/agbd_t_pi_lower_a2"       
 [42] "agbd_prediction/agbd_t_pi_lower_a3"       
 [43] "agbd_prediction/agbd_t_pi_lower_a4"       
 [44] "agbd_prediction/agbd_t_pi_lower_a5"       
 [45] "agbd_prediction/agbd_t_pi_lower_a6"       
 [46] "agbd_prediction/agbd_t_pi_upper_a1"       
 [47] "agbd_prediction/agbd_t_pi_upper_a10"      
 [48] "agbd_prediction/agbd_t_pi_upper_a2"       
 [49] "agbd_prediction/agbd_t_pi_upper_a3"       
 [50] "agbd_prediction/agbd_t_pi_upper_a4"       
 [51] "agbd_prediction/agbd_t_pi_upper_a5"       
 [52] "agbd_prediction/agbd_t_pi_upper_a6"       
 [53] "agbd_prediction/agbd_t_se_a1"             
 [54] "agbd_prediction/agbd_t_se_a10"            
 [55] "agbd_prediction/agbd_t_se_a2"             
 [56] "agbd_prediction/agbd_t_se_a3"             
 [57] "agbd_prediction/agbd_t_se_a4"             
 [58] "agbd_prediction/agbd_t_se_a5"             
 [59] "agbd_prediction/agbd_t_se_a6"             
 [60] "agbd_prediction/algorithm_run_flag_a1"    
 [61] "agbd_prediction/algorithm_run_flag_a10"   
 [62] "agbd_prediction/algorithm_run_flag_a2"    
 [63] "agbd_prediction/algorithm_run_flag_a3"    
 [64] "agbd_prediction/algorithm_run_flag_a4"    
 [65] "agbd_prediction/algorithm_run_flag_a5"    
 [66] "agbd_prediction/algorithm_run_flag_a6"    
 [67] "agbd_prediction/l2_quality_flag_a1"       
 [68] "agbd_prediction/l2_quality_flag_a10"      
 [69] "agbd_prediction/l2_quality_flag_a2"       
 [70] "agbd_prediction/l2_quality_flag_a3"       
 [71] "agbd_prediction/l2_quality_flag_a4"       
 [72] "agbd_prediction/l2_quality_flag_a5"       
 [73] "agbd_prediction/l2_quality_flag_a6"       
 [74] "agbd_prediction/l4_quality_flag_a1"       
 [75] "agbd_prediction/l4_quality_flag_a10"      
 [76] "agbd_prediction/l4_quality_flag_a2"       
 [77] "agbd_prediction/l4_quality_flag_a3"       
 [78] "agbd_prediction/l4_quality_flag_a4"       
 [79] "agbd_prediction/l4_quality_flag_a5"       
 [80] "agbd_prediction/l4_quality_flag_a6"       
 [

ERROR: Error in H5File.open(filename, mode, file_create_pl, file_access_pl): HDF5-API Errors:
    error #000: H5F.c in H5Fopen(): line 836: unable to synchronously open file
        class: HDF5
        major: File accessibility
        minor: Unable to open file

    error #001: H5F.c in H5F__open_api_common(): line 796: unable to open file
        class: HDF5
        major: File accessibility
        minor: Unable to open file

    error #002: H5VLcallback.c in H5VL_file_open(): line 3863: open failed
        class: HDF5
        major: Virtual Object Layer
        minor: Can't open object

    error #003: H5VLcallback.c in H5VL__file_open(): line 3675: open failed
        class: HDF5
        major: Virtual Object Layer
        minor: Can't open object

    error #004: H5VLnative_file.c in H5VL__native_file_open(): line 128: unable to open file
        class: HDF5
        major: File accessibility
        minor: Unable to open file

    error #005: H5Fint.c in H5F_open(): line 1965: un